In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from functions import MLP_generator,test_MLP_generator,createGraphANN
import cPickle

In [2]:
batch_size = 100
seq_len = 16 # same as test set for convenience
learning_rate = 1e-4
epoch = 30

In [3]:
# time dependent list of discrete variables of shape (B,T)
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']
# time independent list of discrete variables of shape of shape (B,)
disc_X = ['store_nbr','item_nbr',
 'family', 'class',
 'perishable', 'city',
 'state', 'type', 'cluster']

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]

In [5]:
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [6]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = 50

In [7]:
lambdas = np.array([2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) * 5e-4 # regularization coefficient
l1,l2,l3 = 5e-4,5e-4,5e-4

** Build Computational Graph **

In [9]:
inputs,train_op,cost,saver,yhat = createGraphANN(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                                        dimentions_X,dimentions_T,dX,dT,d,lambdas,l1,l2,l3)

** Training **

In [10]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
monitor = 10000

In [12]:
prefix = 'train'

In [13]:
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [14]:
prefix = 'val'

In [15]:
y_np2 = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np2 = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np2 = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np2 = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np2 = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np2 = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [16]:
for i in range(epoch):
    cost_train = .0
    weight_train = .0
    cost_val = .0
    weight_val = .0
    for j,X_nps in enumerate(MLP_generator(y_np, weight_np,Con_np,Dis_np,X_np,Count_np,\
                              batch_size,seq_len,shuffle=True,downSample=0.5)):
        _,cost_np = sess.run([train_op,cost],dict(zip(inputs,X_nps+[learning_rate])))
        cost_train += cost_np
        weight_train = weight_train + np.sum(X_nps[1])
        
        if j%monitor == 0:
            for X_nps in MLP_generator(y_np2, weight_np2,Con_np2,Dis_np2,X_np2,Count_np2,\
                                       batch_size,seq_len,shuffle=False,downSample=0.5):
                cost_np = sess.run([cost],dict(zip(inputs,X_nps)))
                cost_val += cost_np[0]       
                weight_val = weight_val + np.sum(X_nps[1])
        
            print "Train loss:{}, Val loss:{}".format(np.sqrt(seq_len*batch_size*cost_train/weight_train),\
                                                      np.sqrt(seq_len*batch_size*cost_val/weight_val))
            cost_train = .0
            weight_train = .0
            cost_val = .0
            weight_val = .0

Train loss:1.72163075426, Val loss:1.80699790054
Train loss:0.826315430584, Val loss:0.914323050991
Train loss:0.905139372221, Val loss:0.920353949345
Train loss:0.940586009691, Val loss:0.9066888186
Train loss:0.813488642837, Val loss:0.897219107901
Train loss:0.76411109071, Val loss:0.886174819001
Train loss:1.03361928655, Val loss:0.882708641081
Train loss:0.725109692347, Val loss:0.876742720611
Train loss:0.889341997174, Val loss:0.870037204655
Train loss:0.711028750132, Val loss:0.861466088819
Train loss:0.691384334558, Val loss:0.848129772192
Train loss:0.664474569449, Val loss:0.829221116824
Train loss:0.597104921712, Val loss:0.809831425283
Train loss:0.793219392997, Val loss:0.797145638729
Train loss:0.713668960322, Val loss:0.789307732898
Train loss:0.554202483128, Val loss:0.784280726634
Train loss:0.536369294145, Val loss:0.782963767628
Train loss:0.570822527581, Val loss:0.783045869187
Train loss:0.723891124296, Val loss:0.779279875493
Train loss:0.650999625192, Val loss:0

In [17]:
# Save model parameters
saver.save(sess,'ANN_fillin_01')

'ANN_fillin_01'

*** Prediction ***

Load trained model

In [18]:
inputs,train_op,cost,saver,yhat = createGraphANN(None,seq_len,cardinalitys_X,cardinalitys_T,\
                                              dimentions_X,dimentions_T,dX,dT,d,lambdas,l1,l2,l3)

In [19]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [20]:
saver.restore(sess,'ANN_fillin_01')

INFO:tensorflow:Restoring parameters from ANN_fillin_01


Load test dataset

In [21]:
prefix = 'test_SI'

In [22]:
y_np_SI = np.loadtxt(prefix+'_Y',dtype=np.int32, delimiter=",") 
Con_np_SI = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_SI = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Dis_np_SI = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [23]:
prefix = 'test_newItem'

In [24]:
y_np_newItem = np.loadtxt(prefix+'_Y',dtype=np.int32, delimiter=",") 
Con_np_newItem = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_newItem = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Dis_np_newItem = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

Score test dataset

In [25]:
yhat_SI, ID_SI = [],[]
for y,x in test_MLP_generator(y_np_SI, Con_np_SI,Dis_np_SI,X_np_SI,batch_size):
    ID_SI.append(np.reshape(y,(-1)))
    yhat_SI.append(np.reshape(sess.run(yhat,dict(zip(inputs[2:-1],x))),(-1)))
SI = pd.DataFrame({'id':np.concatenate(ID_SI),'unit_sales':np.concatenate(yhat_SI)})

In [26]:
yhat_newItem, ID_newItem = [],[]
for y,x in test_MLP_generator(y_np_newItem, Con_np_newItem,Dis_np_newItem,X_np_newItem,batch_size):
    ID_newItem.append(np.reshape(y,(-1)))
    yhat_newItem.append(np.reshape(sess.run(yhat,dict(zip(inputs[2:-1],x))),(-1)))
newItem = pd.DataFrame({'id':np.concatenate(ID_newItem),'unit_sales':np.concatenate(yhat_newItem)})    

In [27]:
zeroSubmission = pd.read_csv('zeroSubmission.csv',dtype={'id': 'int32','unit_sales': 'float32'})

In [28]:
pd.concat([SI,newItem,zeroSubmission]).sort_values(['id']).to_csv('ANN-submit1.csv',index=False)

In [37]:
#submission = pd.read_csv('sample_submission.csv',dtype={'id': 'int32','unit_sales': 'float32'})
#forecastId = SI.id.tolist() + newItem.id.tolist()
#submission[~submission.id.isin(forecastId)].to_csv('zeroSubmission.csv',index=False)